In [1]:
import tensorflow as tf
import numpy as np

### Reading .h5 files

In [2]:
# Create function for mean IOU
class MaskMeanIoU(tf.keras.metrics.MeanIoU):
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.argmax(y_pred, axis=-1)
        return super().update_state(y_true, y_pred, sample_weight=sample_weight)
    
    
THRESHOLD = 0.5
def precision(y_true, y_pred, threshold_shift=0.5-THRESHOLD):
    """
    Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """

    # just in case 
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))

    precision = tp / (tp + fp)
    return precision


def recall(y_true, y_pred, threshold_shift=0.5-THRESHOLD):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    # just in case 
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fn = K.sum(K.round(K.clip(y_true - y_pred_bin, 0, 1)))

    recall = tp / (tp + fn)
    return recall


def fbeta(y_true, y_pred, threshold_shift=0.5-THRESHOLD):
    """Computes the F score.
    The F score is the weighted harmonic mean of precision and recall.
    Here it is only computed as a batch-wise average, not globally.
    This is useful for multi-label classification, where input samples can be
    classified as sets of labels. By only using accuracy (precision) a model
    would achieve a perfect score by simply assigning every class to every
    input. In order to avoid this, a metric should penalize incorrect class
    assignments as well (recall). The F-beta score (ranged from 0.0 to 1.0)
    computes this, as a weighted mean of the proportion of correct class
    assignments vs. the proportion of incorrect class assignments.
    With beta = 1, this is equivalent to a F-measure. With beta < 1, assigning
    correct classes becomes more important, and with beta > 1 the metric is
    instead weighted towards penalizing incorrect class assignments.
    """
    # Beta score. 1.0 = F1 score.
    beta = 1

    # just in case 
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall)

In [3]:
model_dir = '../models/model.h5'
keras_model = tf.keras.models.load_model(model_dir, custom_objects={"MaskMeanIoU": MaskMeanIoU, "precision": precision, "recall": recall, "fbeta": fbeta})


print(keras_model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_image[0][0]                
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
____________________________________________________________________________________________

### Converting to Tensorflow to TFLite model

In [4]:
import pathlib

In [5]:
TFLITE_FILE_PATH = "../models/mobileunet_model.tflite"

converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)

tflite_model = converter.convert()

tflite_model_file = pathlib.Path(TFLITE_FILE_PATH)
tflite_model_file.write_bytes(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/dj/n1wfr2ks7nd1dwjfb8vvqt6h0000gn/T/tmp1h7yhnew/assets


4989376

### Reading TFLite model

In [6]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=TFLITE_FILE_PATH)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [7]:
input_details

[{'name': 'input_image',
  'index': 0,
  'shape': array([  1, 224, 224,   3], dtype=int32),
  'shape_signature': array([ -1, 224, 224,   3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [8]:
output_details

[{'name': 'Identity',
  'index': 188,
  'shape': array([  1, 224, 224,   7], dtype=int32),
  'shape_signature': array([ -1, 224, 224,   7], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

### Converting Tensorflow to CoreML

In [9]:
import coremltools as ct 

In [10]:
mlmodel = ct.convert(keras_model)

Translating MIL ==> MLModel Ops: 100%|██████████| 737/737 [00:00<00:00, 4969.95 ops/s]


In [11]:
#https://coremltools.readme.io/docs/tensorflow-2
 
COREML_FILE_PATH = "../models/mobileunet_model.mlmodel"

    
mlmodel = ct.convert(keras_model,
                    inputs=[ct.ImageType(bias=[-1,-1,-1], scale=1/127)])

mlmodel.save(COREML_FILE_PATH)

Translating MIL ==> MLModel Ops: 100%|██████████| 731/731 [00:00<00:00, 4250.44 ops/s]


### Reading CoreML Model

In [12]:
from coremltools.models import MLModel

loaded_model = MLModel(COREML_FILE_PATH)
loaded_model

input {
  name: "input_image"
  type {
    imageType {
      width: 224
      height: 224
      colorSpace: RGB
      imageSizeRange {
        widthRange {
          lowerBound: 224
          upperBound: 224
        }
        heightRange {
          lowerBound: 224
          upperBound: 224
        }
      }
    }
  }
}
output {
  name: "Identity"
  type {
    multiArrayType {
      dataType: FLOAT32
    }
  }
}
metadata {
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "tensorflow==2.4.0"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "4.1"
  }
}